In [1]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

### Q

The Linear program can be written as follows:\

$\text{Decision variables:}\\ a_i  \text{ -amount(in tons) of alloy A-i from inhouse stock.      i= 1,2,3...10} \\
b_i \text{ -amount (in tons) of alloy A-i purchased from market.     i=1,2,3....10}$\

$\text{Let A= }\ (a_1, a_2,.....,a_{10}) \text{ and B= }\ (b_1, b_2......b_{10})  $

\
Objective:  \
$\text{minimize }\  C^{T}A + M^{T}B$

S.T \

$ \ Ch^{T}(A+B) = 20250\\
Co^{T}(A+B) = 15750\\
S^{T}(A+B) = 9000$\

Where

C = [ 35,  50,  58,  60,  44,  39,  45,  55,  35,  40]

M= [72,  95, 110, 125,  88,  74,  95, 115,  60,  84]

Ch =  [15, 15, 10, 20, 25, 10, 50, 15, 30, 55]

Co = [80, 75, 75, 60, 55, 55, 40, 35, 30, 30]

Si = [5, 10, 15, 20, 20, 35, 10, 50, 40, 15]\



(The constraint that sum of quantities = 450 is included within the 3 constraints since 450 is substituted as the denominator while calculating the relative percentage of each metal in the overall mixture. We can do this since all the constraints have a strict equality sign)



In [2]:
from pyomo.environ import *
import numpy as np

In [3]:
chr_qty = [15, 15, 10, 20, 25, 10, 50, 15, 30, 55]
cop_qty = [80, 75, 75, 60, 55, 55, 40, 35, 30, 30]
sil_qty = [5, 10, 15, 20, 20, 35, 10, 50, 40, 15]

inhouse = [12, 9, 16, 25, 4 ,5 ,10, 13, 0 ,6]
inhouse_cost = [35, 50, 58, 60, 44, 39, 45, 55, 35, 40]

mkt_cost = [72, 95, 110, 125 ,88 ,74, 95, 115 ,60, 84]

In [4]:
cons_rhs = [15750,20250,9000]#,450]

In [5]:
ub = [12, 9 ,16, 25, 4 ,5 ,10, 13, 0, 6,np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]
lb = [0]*20

In [6]:
tcost = np.array(inhouse_cost+mkt_cost)

In [7]:
tcost

array([ 35,  50,  58,  60,  44,  39,  45,  55,  35,  40,  72,  95, 110,
       125,  88,  74,  95, 115,  60,  84])

In [8]:
N = 20
M = 3
colinds = np.arange(N)
rowinds = np.arange(M)


In [9]:
m3 = ConcreteModel()
m3.x = Var(colinds)
m3.cons = ConstraintList()

In [10]:
cons_coeffs = np.array([chr_qty*2,cop_qty*2,sil_qty*2])

In [11]:
for j in range(M):
    m3.cons.add(sum(cons_coeffs[j][i]*m3.x[i] for i in range(N))==cons_rhs[j])

In [12]:
for i in range(N):
    m3.x[i].setlb(lb[i])
    m3.x[i].setub(ub[i])



In [13]:
m3.cost = Objective(expr = sum(m3.x[i]*tcost[i] for i in range(20)), sense = minimize)

In [14]:
m3.pprint()

2 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   20 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

1 Var Declarations
    x : Size=20, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :    12 : False :  True :  Reals
          1 :     0 :  None :     9 : False :  True :  Reals
          2 :     0 :  None :    16 : False :  True :  Reals
          3 :     0 :  None :    25 : False :  True :  Reals
          4 :     0 :  None :     4 : False :  True :  Reals
          5 :     0 :  None :     5 : False :  True :  Reals
          6 :     0 :  None :    10 : False :  True :  Reals
          7 :     0 :  None :    13 : False :  True :  Reals
          8 :     0 :  No

In [15]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(m3).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 31013.4
  Upper bound: 31013.4
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 21
  Number of nonzeros: 61
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.4041469097137451
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

### Q

In [16]:
print('Optimal cost is ',m3.cost())

Optimal cost is  31013.40000000001


In [17]:
for i in range(10):
    print('The value of a%d is'%(i+1), m3.x[i](), 'and value of b%d is'%(i+1), m3.x[i+10]())

The value of a1 is 12.0 and value of b1 is 79.0000000000001
The value of a2 is 9.0 and value of b2 is 0.0
The value of a3 is 16.0 and value of b3 is 0.0
The value of a4 is 25.0 and value of b4 is 0.0
The value of a5 is 4.0 and value of b5 is 0.0
The value of a6 is 5.0 and value of b6 is 0.0
The value of a7 is 10.0 and value of b7 is 0.0
The value of a8 is 0.0 and value of b8 is 0.0
The value of a9 is 0.0 and value of b9 is 120.4
The value of a10 is 6.0 and value of b10 is 163.6


In [18]:
mktdats = sorted(filter(lambda i: i[1]>0, enumerate([m3.x[i+10]() for i in range(10) ])),reverse=True, key = lambda i:i[1])

In [19]:
mktdats

[(9, 163.6), (8, 120.4), (0, 79.0000000000001)]

### Q

In [20]:
print("The 3 Most bought alloys are :")
for i in range(3):
    print('Alloy A-%d, (%.2f tons)'%(mktdats[i][0]+1,mktdats[i][1]))

The 3 Most bought alloys are :
Alloy A-10, (163.60 tons)
Alloy A-9, (120.40 tons)
Alloy A-1, (79.00 tons)


In [21]:
print("The 3 least bought alloys are (from among the ones that needed to be bought) :")
for i in range(1,4):
    print('Alloy A-%d, (%.2f tons)'%(mktdats[-i][0]+1,mktdats[-i][1]))

The 3 least bought alloys are (from among the ones that needed to be bought) :
Alloy A-1, (79.00 tons)
Alloy A-9, (120.40 tons)
Alloy A-10, (163.60 tons)


### Q

In [22]:
print('Stocks of ',end = '')
for i in range(10):
    if m3.x[i]()==m3.x[i].ub:
        print('Alloy A-%d, '%(i+1), end='')
print('have been completely utilised')        
        

Stocks of Alloy A-1, Alloy A-2, Alloy A-3, Alloy A-4, Alloy A-5, Alloy A-6, Alloy A-7, Alloy A-9, Alloy A-10, have been completely utilised


### Q

In [23]:
print('Stocks of ',end = '')
for i in range(11,20):
    if m3.x[i]()==0:
        print('Alloy A-%d, '%(i-9), end='')
print('have not been bought from the market')
print('This is due to the fact that amount of in house stock was sufficient')

Stocks of Alloy A-2, Alloy A-3, Alloy A-4, Alloy A-5, Alloy A-6, Alloy A-7, Alloy A-8, have not been bought from the market
This is due to the fact that amount of in house stock was sufficient
